<a href="https://colab.research.google.com/github/kennenvi/Challange-Dados/blob/main/Analisando_base_de_dados_com_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilizando o Spark no Google Colab

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 53.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=418b6ab5a4ff77493f181a2f41ef56284ad8540c8e9816ce48c015a9ce061f02
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


# Carregamento de Dados
---

## SparkSession

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [3]:
spark

## Baixando dados

In [4]:
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip' && unzip semana-1.zip -d dados/

--2022-12-02 00:42:14--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.83.64, 52.217.66.100, 52.216.28.12, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.83.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip’

semana-1.zip        100%[===================>]  18.10M  40.5MB/s    in 0.4s    

2022-12-02 00:42:14 (40.5 MB/s) - ‘semana-1.zip’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dados/dataset_bruto.json  


# Iniciando análise

In [5]:
# Importando modulos e classes

import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType, IntegerType, StringType

In [6]:
# Lendo dados e transformando-os em um DataFrame

dados = spark.read.json('/content/dados/dataset_bruto.json')

### Avaliando a estrutura dos dados

In [7]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [8]:
print(f'Quantidade de linhas = {dados.count()}, Quantidade de colunas {len(dados.columns)}')

Quantidade de linhas = 89083, Quantidade de colunas 3


### Selecionando os Subcampos da caluna anúncio

Porque como o objetivo é prever um valor para o imóvel apenas os dados da coluna anúncio serão utilizados

In [9]:
dados = dados.select('anuncio.*')

In [10]:
dados.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

## Convertendo colunas para tipos correspondetes

Algumas colunas parecem estar em formato array porém possuem apenas um elemento. 
Por isso iremos verificar se isso se aplica a coluna inteira

In [12]:
# Colunas com dados do tipo inteiro, porém em arrays
colunas_array = ['area_total', 'area_util', 'banheiros', 'quartos', 'suites', 'vaga']

In [14]:
# Verificando frequência do tamanho de cada array das colunas em "colunas_array"
for col in colunas_array:
    dados\
        .select(f.size(f.col(col)).alias(col))\
        .groupby(col)\
        .count()\
        .show()

+----------+-----+
|area_total|count|
+----------+-----+
|         1|76488|
|         0|12595|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|88846|
|        2|  224|
|        0|   13|
+---------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|88798|
|        2|  112|
|        0|  173|
+---------+-----+

+-------+-----+
|quartos|count|
+-------+-----+
|      1|88120|
|      2|  160|
|      0|  803|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|79431|
|     2|  103|
|     0| 9549|
+------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|84037|
|   2|   94|
|   0| 4952|
+----+-----+



Por causa de a maioria conter apenas um valor será utilizado o primeiro valor para essas colunas

In [46]:
# Retirando os dados de arrays e transformando-os em inteiros
dados = dados\
    .select([f.col(col)[0].cast(IntegerType()).alias(col) if col in  colunas_array else col for col in dados.columns])

0.05735525091999534

In [14]:
# Coluna com arrays porém do tipo string
dados = dados.withColumn('caracteristicas', dados['caracteristicas'][0])

In [15]:
dados.show()

+-----+----------+---------+---------+-----------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|  caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+-----------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|             null|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|             null|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|             null|{Maria da Graça, ...|1fb030a5-9e3e-4a1...|      0|  nu

## Filtrando a base de dados

Verificando frêquencia das colunas "tipo_anuncio", "tipo_unidade", "tipo_uso"

In [16]:
dados\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [17]:
dados\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [18]:
dados\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



Selecionando apenas as categorias com mais valores, pois estes representam uma parcela maior do conjunto de dados

In [19]:
dados = dados.where("tipo_uso = 'Residencial' and tipo_unidade = 'Apartamento' and tipo_anuncio = 'Usado'")

## Extraindo bairro e zona da coluna endereco

In [20]:
# Selecionando os campos
dados = dados.withColumn('bairro', dados['endereco.bairro'])
dados = dados.withColumn('zona', dados['endereco.zona'])

In [21]:
# Excluindo coluna endereco
dados = dados.drop('endereco')

In [22]:
dados.show()

+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|   caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|          Academia|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|     Churrasqueira|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|      Portar

## Transformando os dados da coluna valores em novas colunas separadas

In [23]:
# Conferindo o esquema dos dados 
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



In [24]:
# Retirando Struct dos arrays
dados = dados\
    .withColumn('valores', f.explode('valores'))

In [25]:
# Selecionando colunas de valores
colunas_valores = dados.select('valores.*').columns

In [26]:
# Adicioando as subcolunas de valores no DataFrame dados
for col in colunas_valores:
    dados = dados.withColumn(col, dados[f'valores.{col}'])

In [27]:
# Excluindo valores
dados = dados.drop('valores')

In [28]:
dados.show()

+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|   caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|          Academia|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|           Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|     Churrasqueira|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|           Paciência|Zona Oeste|         0|   0|Venda|15000|
|    1|        

## Selecionando apenas amostras cujo "tipo" seja Venda

In [29]:
dados = dados\
    .where('tipo = "Venda"')

# Salvando os dados

Salvando dados em formato parquet

In [1]:
dados.write\
        .parquet(
            'resultados/dados_processados_parquet',
            mode='overwrite'
            )

NameError: ignored

Salvando dados em formato csv

In [ ]:
dados.write\
        .csv(
            'resultados/dados_processados_csv',
            mode='overwrite',
            header=True
            )

Salvando dados em formato ORC

In [ ]:
dados.write.orc(
    'resultados/dados_processados_orc',
    mode='overwrite',    
)

### Comparando desempenho de leitura

In [ ]:
%%time
spark.read.parquet(
    '/content/drive/Othercomputers/Fino do Fino 2.0/Python Notebook/Challange_Data_Science/Semana01/dados_processados_parquet'
)

In [ ]:
%%time
spark.read.csv(
    '/content/drive/Othercomputers/Fino do Fino 2.0/Python Notebook/Challange_Data_Science/Semana01/dados_processados_csv',
    inferSchema=True,
    header=True
)

In [ ]:
%%time
spark.read.orc(
    '/content/drive/Othercomputers/Fino do Fino 2.0/Python Notebook/Challange_Data_Science/Semana01/dados_processados_orc'
)

O desempenho de leitura de formatos parquet se mostrou mais rápido que a leitura de csv e de ORC